In [ ]:
%matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 19: Not Enough Minerals ---</h2><p>Your scans show that the lava did indeed form obsidian!</p>
<p>The wind has changed direction enough to stop sending lava droplets toward you, so you and the elephants exit the cave. As you do, you notice a collection of <a href="https://en.wikipedia.org/wiki/Geode" target="_blank">geodes</a> around the pond. Perhaps you could use the obsidian to create some <em>geode-cracking robots</em> and break them open?</p>
<p>To collect the obsidian from the bottom of the pond, you'll need waterproof <em>obsidian-collecting robots</em>. Fortunately, there is an abundant amount of clay nearby that you can use to make them waterproof.</p>
<p>In order to harvest the clay, you'll need special-purpose <em>clay-collecting robots</em>. To make any type of robot, you'll need <em>ore</em>, which is also plentiful but in the opposite direction from the clay.</p>
<p>Collecting ore requires <em>ore-collecting robots</em> with big drills. Fortunately, <em>you have exactly one ore-collecting robot</em> in your pack that you can use to <span title="If You Give A Mouse An Ore-Collecting Robot">kickstart</span> the whole operation.</p>
<p>Each robot can collect 1 of its resource type per minute. It also takes one minute for the robot factory (also conveniently from your pack) to construct any type of robot, although it consumes the necessary resources available when construction begins.</p>
<p>The robot factory has many <em>blueprints</em> (your puzzle input) you can choose from, but once you've configured it with a blueprint, you can't change it. You'll need to work out which blueprint is best.</p>
<p>For example:</p>
<pre><code>Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.

Blueprint 2:
Each ore robot costs 2 ore.
Each clay robot costs 3 ore.
Each obsidian robot costs 3 ore and 8 clay.
Each geode robot costs 3 ore and 12 obsidian.
</code></pre>

<p>(Blueprints have been line-wrapped here for legibility. The robot factory's actual assortment of blueprints are provided one blueprint per line.)</p>
<p>The elephants are starting to look hungry, so you shouldn't take too long; you need to figure out which blueprint would maximize the number of opened geodes after <em>24 minutes</em> by figuring out which robots to build and when to build them.</p>
<p>Using blueprint 1 in the example above, the largest number of geodes you could open in 24 minutes is <code><em>9</em></code>. One way to achieve that is:</p>
<pre><code>== Minute 1 ==
1 ore-collecting robot collects 1 ore; you now have 1 ore.

== Minute 2 ==
1 ore-collecting robot collects 1 ore; you now have 2 ore.

== Minute 3 ==
Spend 2 ore to start building a clay-collecting robot.
1 ore-collecting robot collects 1 ore; you now have 1 ore.
The new clay-collecting robot is ready; you now have 1 of them.

== Minute 4 ==
1 ore-collecting robot collects 1 ore; you now have 2 ore.
1 clay-collecting robot collects 1 clay; you now have 1 clay.

== Minute 5 ==
Spend 2 ore to start building a clay-collecting robot.
1 ore-collecting robot collects 1 ore; you now have 1 ore.
1 clay-collecting robot collects 1 clay; you now have 2 clay.
The new clay-collecting robot is ready; you now have 2 of them.

== Minute 6 ==
1 ore-collecting robot collects 1 ore; you now have 2 ore.
2 clay-collecting robots collect 2 clay; you now have 4 clay.

== Minute 7 ==
Spend 2 ore to start building a clay-collecting robot.
1 ore-collecting robot collects 1 ore; you now have 1 ore.
2 clay-collecting robots collect 2 clay; you now have 6 clay.
The new clay-collecting robot is ready; you now have 3 of them.

== Minute 8 ==
1 ore-collecting robot collects 1 ore; you now have 2 ore.
3 clay-collecting robots collect 3 clay; you now have 9 clay.

== Minute 9 ==
1 ore-collecting robot collects 1 ore; you now have 3 ore.
3 clay-collecting robots collect 3 clay; you now have 12 clay.

== Minute 10 ==
1 ore-collecting robot collects 1 ore; you now have 4 ore.
3 clay-collecting robots collect 3 clay; you now have 15 clay.

== Minute 11 ==
Spend 3 ore and 14 clay to start building an obsidian-collecting robot.
1 ore-collecting robot collects 1 ore; you now have 2 ore.
3 clay-collecting robots collect 3 clay; you now have 4 clay.
The new obsidian-collecting robot is ready; you now have 1 of them.

== Minute 12 ==
Spend 2 ore to start building a clay-collecting robot.
1 ore-collecting robot collects 1 ore; you now have 1 ore.
3 clay-collecting robots collect 3 clay; you now have 7 clay.
1 obsidian-collecting robot collects 1 obsidian; you now have 1 obsidian.
The new clay-collecting robot is ready; you now have 4 of them.

== Minute 13 ==
1 ore-collecting robot collects 1 ore; you now have 2 ore.
4 clay-collecting robots collect 4 clay; you now have 11 clay.
1 obsidian-collecting robot collects 1 obsidian; you now have 2 obsidian.

== Minute 14 ==
1 ore-collecting robot collects 1 ore; you now have 3 ore.
4 clay-collecting robots collect 4 clay; you now have 15 clay.
1 obsidian-collecting robot collects 1 obsidian; you now have 3 obsidian.

== Minute 15 ==
Spend 3 ore and 14 clay to start building an obsidian-collecting robot.
1 ore-collecting robot collects 1 ore; you now have 1 ore.
4 clay-collecting robots collect 4 clay; you now have 5 clay.
1 obsidian-collecting robot collects 1 obsidian; you now have 4 obsidian.
The new obsidian-collecting robot is ready; you now have 2 of them.

== Minute 16 ==
1 ore-collecting robot collects 1 ore; you now have 2 ore.
4 clay-collecting robots collect 4 clay; you now have 9 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 6 obsidian.

== Minute 17 ==
1 ore-collecting robot collects 1 ore; you now have 3 ore.
4 clay-collecting robots collect 4 clay; you now have 13 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 8 obsidian.

== Minute 18 ==
Spend 2 ore and 7 obsidian to start building a geode-cracking robot.
1 ore-collecting robot collects 1 ore; you now have 2 ore.
4 clay-collecting robots collect 4 clay; you now have 17 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 3 obsidian.
The new geode-cracking robot is ready; you now have 1 of them.

== Minute 19 ==
1 ore-collecting robot collects 1 ore; you now have 3 ore.
4 clay-collecting robots collect 4 clay; you now have 21 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 5 obsidian.
1 geode-cracking robot cracks 1 geode; you now have 1 open geode.

== Minute 20 ==
1 ore-collecting robot collects 1 ore; you now have 4 ore.
4 clay-collecting robots collect 4 clay; you now have 25 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 7 obsidian.
1 geode-cracking robot cracks 1 geode; you now have 2 open geodes.

== Minute 21 ==
Spend 2 ore and 7 obsidian to start building a geode-cracking robot.
1 ore-collecting robot collects 1 ore; you now have 3 ore.
4 clay-collecting robots collect 4 clay; you now have 29 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 2 obsidian.
1 geode-cracking robot cracks 1 geode; you now have 3 open geodes.
The new geode-cracking robot is ready; you now have 2 of them.

== Minute 22 ==
1 ore-collecting robot collects 1 ore; you now have 4 ore.
4 clay-collecting robots collect 4 clay; you now have 33 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 4 obsidian.
2 geode-cracking robots crack 2 geodes; you now have 5 open geodes.

== Minute 23 ==
1 ore-collecting robot collects 1 ore; you now have 5 ore.
4 clay-collecting robots collect 4 clay; you now have 37 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 6 obsidian.
2 geode-cracking robots crack 2 geodes; you now have 7 open geodes.

== Minute 24 ==
1 ore-collecting robot collects 1 ore; you now have 6 ore.
4 clay-collecting robots collect 4 clay; you now have 41 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 8 obsidian.
2 geode-cracking robots crack 2 geodes; you now have 9 open geodes.
</code></pre>

<p>However, by using blueprint 2 in the example above, you could do even better: the largest number of geodes you could open in 24 minutes is <code><em>12</em></code>.</p>
<p>Determine the <em>quality level</em> of each blueprint by <em>multiplying that blueprint's ID number</em> with the largest number of geodes that can be opened in 24 minutes using that blueprint. In this example, the first blueprint has ID 1 and can open 9 geodes, so its quality level is <code><em>9</em></code>. The second blueprint has ID 2 and can open 12 geodes, so its quality level is <code><em>24</em></code>. Finally, if you <em>add up the quality levels</em> of all of the blueprints in the list, you get <code><em>33</em></code>.</p>
<p>Determine the quality level of each blueprint using the largest number of geodes it could produce in 24 minutes. <em>What do you get if you add up the quality level of all of the blueprints in your list?</em></p>
</article>


In [ ]:
from collections import deque
from pprint import pprint


class BluePrint(Print):
    def __init__(self, s: str) -> None:
        data = (int(i) for i in re.findall(r"\d+", s))

        self.id = next(data)
        self.ore_cost_ore_robot = next(data)
        self.ore_cost_clay_robot = next(data)
        self.ore_cost_obsidian_robot = next(data)
        self.clay_cost_obsidian_robot = next(data)
        self.ore_cost_geode_robot = next(data)
        self.obsidian_cost_geode_robot = next(data)

    def max_number_geodes(self, minutes: int) -> int:
        seen = {}
        queue = deque([(0, 0, 0, 0, 0, 1, 0, 0, 0)])

        while queue:
            minute, *state = queue.popleft()

            (
                ore,
                clay,
                obsidian,
                geode,
                ore_robot,
                clay_robot,
                obsidian_robot,
                geode_robot,
            ) = state

            state = tuple(state)

            if state in seen:
                continue

            seen[state] = geode

            if minute == minutes:
                continue

            if (
                ore >= self.ore_cost_geode_robot
                and obsidian >= self.obsidian_cost_geode_robot
            ):
                queue.append(
                    (
                        minute + 1,
                        ore + ore_robot - self.ore_cost_geode_robot,
                        clay + clay_robot,
                        obsidian + obsidian_robot - self.obsidian_cost_geode_robot,
                        geode + geode_robot,
                        ore_robot,
                        clay_robot,
                        obsidian_robot,
                        geode_robot + 1,
                    )
                )

            if (
                ore >= self.ore_cost_obsidian_robot
                and clay >= self.clay_cost_obsidian_robot
            ):
                queue.append(
                    (
                        minute + 1,
                        ore + ore_robot - self.ore_cost_obsidian_robot,
                        clay + clay_robot - self.clay_cost_obsidian_robot,
                        obsidian + obsidian_robot,
                        geode + geode_robot,
                        ore_robot,
                        clay_robot,
                        obsidian_robot + 1,
                        geode_robot,
                    )
                )

            if ore >= self.ore_cost_clay_robot:
                queue.append(
                    (
                        minute + 1,
                        ore + ore_robot - self.ore_cost_clay_robot,
                        clay + clay_robot,
                        obsidian + obsidian_robot,
                        geode + geode_robot,
                        ore_robot,
                        clay_robot + 1,
                        obsidian_robot,
                        geode_robot,
                    )
                )

            if ore >= self.ore_cost_ore_robot:
                queue.append(
                    (
                        minute + 1,
                        ore + ore_robot - self.ore_cost_ore_robot,
                        clay + clay_robot,
                        obsidian + obsidian_robot,
                        geode + geode_robot,
                        ore_robot + 1,
                        clay_robot,
                        obsidian_robot,
                        geode_robot,
                    )
                )

            queue.append(
                (
                    minute + 1,
                    ore + ore_robot,
                    clay + clay_robot,
                    obsidian + obsidian_robot,
                    geode + geode_robot,
                    ore_robot,
                    clay_robot,
                    obsidian_robot,
                    geode_robot,
                )
            )

        return max(seen.values())


class Collector(Print):
    def __init__(self, s: str) -> None:
        self.blueprints = [BluePrint(l.strip()) for l in s.strip().splitlines()]

    def quality_level_of_all_blueprints(self, minutes=24) -> int:
        # return sum(bp.id * bp.max_number_geodes(minutes) for bp in self.blueprints)
        return self.blueprints[0].max_number_geodes(minutes)


tests = [
    {
        "name": "Example 1",
        "s": """
            Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
            Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.
        """,
        "expected": 33,
    },
]


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Collector(s).quality_level_of_all_blueprints()

In [ ]:
with open("../input/day19.txt") as quotient:
    puzzle = quotient.read()

print(f"Part I: {Collector(puzzle).quality_level_of_all_blueprints()}")

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4450</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2>


In [ ]:
# print(f"Part II: {CubesII(puzzle).exterior_surface_area():,}")

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>2564</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
